In [2]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[box2d]")
easyinstall("bbrl_gymnasium[classic_control]")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import sys
from pathlib import Path
import math

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

from omegaconf import OmegaConf
import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime
OmegaConf.register_new_resolver(
    "current_time", lambda: strftime("%Y%m%d-%H%M%S"), replace=True
)
from bbrl.agents.gymnasium import GymAgent, ParallelGymAgent, make_env, record_video

In [4]:
from datetime import datetime
from datetime import date

import base64
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium
from gymnasium import logger as gymlogger
# from gym.wrappers import Monitor # deprecated 2023 - https://stackoverflow.com/questions/71520568/importerror-cannot-import-name-monitor-from-gym-wrappers
from gymnasium.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")

def show_video(loop=True, num=0):
    mp4list = glob.glob(f'videoTest/rl-video-episode-{num}.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if loop == True:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
        else:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    
def wrap_env(env):
    env = RecordVideo(env, './videoTest',  episode_trigger = lambda episode_number: True) # !!! 2023
    env.reset() # !!! 2023
    #env = Monitor(env, './video', force=True)
    return env


 2024-01-12 11:32:26 GMT
OK.


In [45]:
"""
Continuous action version of the classic cart-pole system implemented by Rich
Sutton et al.
"""
import math

import numpy as np

from gymnasium.envs.classic_control.cartpole import CartPoleEnv
from gymnasium import logger, spaces
from gymnasium.wrappers import TimeLimit

import sys
sys.modules[__name__]

class ContinuousEnvCACLA(CartPoleEnv):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1
        self.max_action = 1
        self.action_space = spaces.Box(
            self.min_action, self.max_action, shape=(1,), dtype=np.float64
        )
        

    def step(self, action):
        if action > self.max_action:
            action = np.array(self.max_action)
        elif action < self.min_action:
            action = np.array(self.min_action)
        assert self.state is not None, "Call reset before using step method."

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * float(action)

        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = -1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

try:
  gymnasium.envs.register(
      id='CartpoleEnvCacla',
      entry_point='__main__:ContinuousEnvCACLA',
      max_episode_steps=500
  )
except:
    print("Except")
    pass



In [58]:
import torch.nn
import numpy as np
from torch.optim import SGD, Adam
class Gav(nn.Module):
    def __init__(self, in_dim, h_dim, activation=nn.Tanh, discount_factor=0.95, gaussian_noise=0.01, lrate = 0.01, decay=0.99, exploration="gaussian"):
        super(Gav,self).__init__()
        self.critic = nn.Sequential(
            nn.Linear(in_dim+1, h_dim),
            activation(),
            nn.Linear(h_dim, 1)
        )
        self.critic_optim = Adam(self.critic.parameters(), lr=lrate)
        
        self.actor = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
            activation()
        )
        self.actor_optim = Adam(self.actor.parameters(), lr=lrate)
        
        self.discount_factor = discount_factor
        self.gaussian_noise = gaussian_noise
        self.e = 1
        self.prob_update = 1
        self.decay_rate = decay
        self.exploration = exploration
    
    def forward(self, x):
        x = torch.from_numpy(x).unsqueeze(0)
        # le critic prend aussi l'action
        act = self.actor(x)
        act = normalize(act)
        # gaussian exploration
        if self.exploration=="gaussian": act = self.sample(act)
        # egreedy exploration
        elif self.exploration=="egreedy" and torch.rand(1) < self.e: act = self.greed()
        x = torch.hstack((x,act))
        return self.critic(x), act
    
    def sample(self, x):
        return (x + np.random.normal(0,self.gaussian_noise))
    
    # nombre entre -1 et 1
    def greed(self):
        return (2 * torch.rand(1) - 1).unsqueeze(0)
    
    def decaye(self):
        self.e*=0.999
        self.e = max(0.01,self.e)
    
    def decay(self):
        self.prob_update*= self.decay_rate
        self.prob_update = max(0.01, self.prob_update)

        
def normalize(x):
    
    #normact = 2*((x+0.91351861)/(0.75200462 + 0.91351861))-1
    # bornes article
    #normact = 2*((x+2.70117164)/(2.76748848 + 2.70117164))-1
    normact = 2*((x+6)/(6 + 6))-1
    #print("Norm act ",normact)
    return normact

def normalize_obs(x):
    
    #bornes = np.array([2.44753194, -2.45538521, 0.2296841, -0.24339156, 2.99952817, -3.0505054, 1.55276, -2.181077])
    
    # bornes pour l'environnement de l'article
    #bornes = np.array([1.22236145, -1.22290611,  0.41785294, -0.47710252,  3.41861892, -2.79673457, 3.13325334, -4.49813938])
    bornes = np.array([4.8, -4.8,  2, -2, 4.1887903e-01, -4.1887903e-01, 2, -2])
    res = np.zeros(4, dtype=np.float32)
    #print("before ",x)
    res[0] = 2*((x[0]-bornes[1])/(bornes[0]-bornes[1]))-1
    res[1] = 2*((x[1]-bornes[3])/(bornes[2]-bornes[3]))-1
    res[2] = 2*((x[2]-bornes[5])/(bornes[4]-bornes[5]))-1
    res[3] = 2*((x[3]-bornes[7])/(bornes[6]-bornes[7]))-1
    
    #print(res[1])
    
    #print("res after ",res)
    return res

def normalize_critic(x):
    
    #return 2*((x+0.19350152)/(11.54955292 + 0.19350152))-1
    # bornes article
    #return 2*((x+8.18684864)/(6.95543098 + 8.18684864))-1
    return 2*((x+50)/(50 + 50))-1

def test(eval_env, model, n_test = 10):
    cum_reward = 0
    with torch.no_grad():
        for _ in range(n_test):
            done, truncated = False, False
            obs0,_ = eval_env.reset()
            while not done and not truncated:
                #obs0 = normalize_obs(obs0)
                _, a0 = model(obs0)  
                obs0, reward, done,truncated,_ = eval_env.step(normalize(a0))
                #obs0, reward, done,truncated,_ = eval_env.step(a0)
                cum_reward += reward
    return cum_reward/n_test
    
def train(eval_env, train_env, model, eval_interval=5000, step_max=100000, exploration="gaussian"):
    it = 0
    obs,_ = train_env.reset()
    p = 1
    scores = np.array([0])
    while it < step_max:
        it += 1
        #obs = normalize_obs(obs)
        model.critic_optim.zero_grad()
            
        q0, action = model(obs)
        #q0 = normalize_critic(q0)
        
        
        obs, reward, done, truncated, reste = train_env.step(action+np.random.normal(0,0.3)) 
        #obs = normalize_obs(obs)
        reward = model.sample(reward+np.random.normal(0,0.3))
        q1, _ = model(obs)
        #q1 = normalize_critic(q1)
        delta = reward + model.discount_factor*q1.detach() - q0
        (delta**2).backward()
        model.critic_optim.step()
        if torch.rand(1) > model.prob_update: 
            model.actor_optim.zero_grad()
            (-model(obs)[0]).backward()
            model.actor_optim.step()
        model.decay()
        # pour diminuer exponentiellement
        model.decaye()
        
        if done or truncated: obs,_ = train_env.reset()
        if it % eval_interval == 0:
            perf = test(eval_env, model)
            print(f'{it = } | reward {perf}')
            scores = np.hstack((scores,perf))
    return scores

In [59]:

# Initialisation
n_runs = 20
final = []
explo = "gaussian"
for run in range(n_runs):
    train_env = gymnasium.make('CartpoleEnvCacla')
    train_env = TimeLimit(train_env, max_episode_steps=500)
    eval_env = gymnasium.make('CartpoleEnvCacla')
    eval_env = TimeLimit(eval_env, max_episode_steps=500)
    print(f'======= {run = } / {n_runs-1} =========')
    gav = Gav(in_dim=4, h_dim=12, 
              discount_factor=0.95, 
              gaussian_noise=0.1,exploration =explo)
    scores = train(eval_env, train_env, gav, eval_interval=1024, step_max=102400)
    print(scores.shape)
    final.append(scores)

final = np.array(final)
np.savetxt("gav"+explo+".txt",final)
    

======= run = 0 / 19 =========
it = 1024 | reward 36.3
it = 2048 | reward 44.8
it = 3072 | reward 47.4
it = 4096 | reward 38.1
it = 5120 | reward 53.4
it = 6144 | reward 79.1
it = 7168 | reward 48.5
it = 8192 | reward 49.1
it = 9216 | reward 204.5
it = 10240 | reward 132.7
it = 11264 | reward 62.3
it = 12288 | reward 62.0
it = 13312 | reward 48.9
it = 14336 | reward 42.1
it = 15360 | reward 51.1
it = 16384 | reward 42.6
it = 17408 | reward 159.2
it = 18432 | reward 133.4
it = 19456 | reward 53.0
it = 20480 | reward 143.4
it = 21504 | reward 140.1
it = 22528 | reward 38.4
it = 23552 | reward 58.3
it = 24576 | reward 47.8
it = 25600 | reward 182.4
it = 26624 | reward 234.1
it = 27648 | reward 43.1
it = 28672 | reward 47.4
it = 29696 | reward 63.7
it = 30720 | reward 199.9
it = 31744 | reward 44.5
it = 32768 | reward 43.3
it = 33792 | reward 178.8
it = 34816 | reward 43.6
it = 35840 | reward 50.9
it = 36864 | reward 55.6
it = 37888 | reward 56.1
it = 38912 | reward 320.9
it = 39936 | rewa

it = 21504 | reward 88.5
it = 22528 | reward 48.7
it = 23552 | reward 214.4
it = 24576 | reward 147.3
it = 25600 | reward 47.3
it = 26624 | reward 49.0
it = 27648 | reward 58.4
it = 28672 | reward 40.3
it = 29696 | reward 53.1
it = 30720 | reward 331.5
it = 31744 | reward 45.4
it = 32768 | reward 48.4
it = 33792 | reward 62.6
it = 34816 | reward 61.2
it = 35840 | reward 169.8
it = 36864 | reward 217.1
it = 37888 | reward 49.8
it = 38912 | reward 40.6
it = 39936 | reward 50.7
it = 40960 | reward 42.1
it = 41984 | reward 119.4
it = 43008 | reward 52.2
it = 44032 | reward 50.0
it = 45056 | reward 47.7
it = 46080 | reward 53.3
it = 47104 | reward 51.8
it = 48128 | reward 143.3
it = 49152 | reward 218.8
it = 50176 | reward 85.7
it = 51200 | reward 49.8
it = 52224 | reward 48.4
it = 53248 | reward 135.7
it = 54272 | reward 319.4
it = 55296 | reward 221.9
it = 56320 | reward 167.5
it = 57344 | reward 97.5
it = 58368 | reward 48.3
it = 59392 | reward 69.6
it = 60416 | reward 52.5
it = 61440 | 

it = 40960 | reward 47.6
it = 41984 | reward 56.0
it = 43008 | reward 42.4
it = 44032 | reward 187.0
it = 45056 | reward 120.9
it = 46080 | reward 72.1
it = 47104 | reward 60.9
it = 48128 | reward 196.4
it = 49152 | reward 58.0
it = 50176 | reward 115.6
it = 51200 | reward 146.6
it = 52224 | reward 62.8
it = 53248 | reward 110.3
it = 54272 | reward 46.7
it = 55296 | reward 46.3
it = 56320 | reward 47.0
it = 57344 | reward 53.6
it = 58368 | reward 78.8
it = 59392 | reward 50.7
it = 60416 | reward 201.8
it = 61440 | reward 70.4
it = 62464 | reward 47.0
it = 63488 | reward 149.8
it = 64512 | reward 59.2
it = 65536 | reward 49.9
it = 66560 | reward 271.3
it = 67584 | reward 271.6
it = 68608 | reward 54.0
it = 69632 | reward 51.1
it = 70656 | reward 51.2
it = 71680 | reward 225.3
it = 72704 | reward 135.0
it = 73728 | reward 240.0
it = 74752 | reward 52.3
it = 75776 | reward 287.6
it = 76800 | reward 357.3
it = 77824 | reward 211.1
it = 78848 | reward 192.7
it = 79872 | reward 50.1
it = 808

it = 61440 | reward 52.5
it = 62464 | reward 171.1
it = 63488 | reward 41.6
it = 64512 | reward 82.3
it = 65536 | reward 68.6
it = 66560 | reward 209.7
it = 67584 | reward 104.2
it = 68608 | reward 180.2
it = 69632 | reward 47.2
it = 70656 | reward 53.1
it = 71680 | reward 131.1
it = 72704 | reward 57.1
it = 73728 | reward 54.5
it = 74752 | reward 52.5
it = 75776 | reward 53.7
it = 76800 | reward 48.9
it = 77824 | reward 38.4
it = 78848 | reward 56.3
it = 79872 | reward 44.7
it = 80896 | reward 51.6
it = 81920 | reward 45.1
it = 82944 | reward 45.8
it = 83968 | reward 52.2
it = 84992 | reward 114.3
it = 86016 | reward 54.2
it = 87040 | reward 41.4
it = 88064 | reward 49.1
it = 89088 | reward 37.6
it = 90112 | reward 47.5
it = 91136 | reward 43.3
it = 92160 | reward 51.9
it = 93184 | reward 48.4
it = 94208 | reward 48.3
it = 95232 | reward 168.2
it = 96256 | reward 68.3
it = 97280 | reward 55.1
it = 98304 | reward 57.6
it = 99328 | reward 53.3
it = 100352 | reward 68.1
it = 101376 | rew

it = 82944 | reward 101.9
it = 83968 | reward 63.9
it = 84992 | reward 67.3
it = 86016 | reward 37.3
it = 87040 | reward 51.2
it = 88064 | reward 315.1
it = 89088 | reward 85.4
it = 90112 | reward 49.1
it = 91136 | reward 279.9
it = 92160 | reward 188.7
it = 93184 | reward 52.4
it = 94208 | reward 56.6
it = 95232 | reward 54.1
it = 96256 | reward 191.4
it = 97280 | reward 64.6
it = 98304 | reward 189.3
it = 99328 | reward 55.6
it = 100352 | reward 56.7
it = 101376 | reward 48.9
it = 102400 | reward 111.7
(101,)
======= run = 13 / 19 =========
it = 1024 | reward 39.6
it = 2048 | reward 58.3
it = 3072 | reward 38.9
it = 4096 | reward 42.1
it = 5120 | reward 33.9
it = 6144 | reward 41.4
it = 7168 | reward 36.8
it = 8192 | reward 36.7
it = 9216 | reward 34.6
it = 10240 | reward 55.0
it = 11264 | reward 44.9
it = 12288 | reward 46.8
it = 13312 | reward 42.8
it = 14336 | reward 38.8
it = 15360 | reward 34.0
it = 16384 | reward 34.6
it = 17408 | reward 34.4
it = 18432 | reward 36.6
it = 19456

it = 102400 | reward 206.6
(101,)
======= run = 16 / 19 =========
it = 1024 | reward 34.4
it = 2048 | reward 45.7
it = 3072 | reward 39.3
it = 4096 | reward 38.0
it = 5120 | reward 39.1
it = 6144 | reward 43.6
it = 7168 | reward 37.4
it = 8192 | reward 38.1
it = 9216 | reward 36.6
it = 10240 | reward 34.2
it = 11264 | reward 36.2
it = 12288 | reward 38.6
it = 13312 | reward 39.0
it = 14336 | reward 38.5
it = 15360 | reward 42.0
it = 16384 | reward 38.8
it = 17408 | reward 32.1
it = 18432 | reward 34.6
it = 19456 | reward 37.9
it = 20480 | reward 42.0
it = 21504 | reward 38.1
it = 22528 | reward 62.2
it = 23552 | reward 53.4
it = 24576 | reward 41.7
it = 25600 | reward 47.8
it = 26624 | reward 63.2
it = 27648 | reward 54.3
it = 28672 | reward 53.4
it = 29696 | reward 51.2
it = 30720 | reward 69.2
it = 31744 | reward 57.2
it = 32768 | reward 67.0
it = 33792 | reward 46.4
it = 34816 | reward 169.8
it = 35840 | reward 40.1
it = 36864 | reward 40.8
it = 37888 | reward 49.8
it = 38912 | rewa

it = 20480 | reward 44.6
it = 21504 | reward 65.4
it = 22528 | reward 201.1
it = 23552 | reward 99.0
it = 24576 | reward 115.1
it = 25600 | reward 48.7
it = 26624 | reward 124.6
it = 27648 | reward 48.4
it = 28672 | reward 129.4
it = 29696 | reward 47.5
it = 30720 | reward 82.5
it = 31744 | reward 76.6
it = 32768 | reward 100.9
it = 33792 | reward 80.1
it = 34816 | reward 50.7
it = 35840 | reward 49.5
it = 36864 | reward 47.0
it = 37888 | reward 50.2
it = 38912 | reward 47.0
it = 39936 | reward 44.8
it = 40960 | reward 89.5
it = 41984 | reward 180.8
it = 43008 | reward 179.4
it = 44032 | reward 238.6
it = 45056 | reward 274.3
it = 46080 | reward 171.3
it = 47104 | reward 64.7
it = 48128 | reward 196.6
it = 49152 | reward 98.7
it = 50176 | reward 50.5
it = 51200 | reward 50.2
it = 52224 | reward 68.1
it = 53248 | reward 75.8
it = 54272 | reward 291.1
it = 55296 | reward 224.9
it = 56320 | reward 235.3
it = 57344 | reward 275.6
it = 58368 | reward 44.1
it = 59392 | reward 108.3
it = 6041

In [25]:
rec_env = wrap_env(TimeLimit(ContinuousEnvCACLA(render_mode='rgb_array'),max_episode_steps=500))
observation, _ = rec_env.reset()
maxEvaluations = 2
evaluation = 0
score = []
total = 0
while evaluation < maxEvaluations:
    rec_env.render()
    _, a0 = gav(observation)  
    action = a0
    observation, reward, done, truncated, _ = rec_env.step(action) 
    total += reward
    if done or truncated:
      score.append(total)
      total = 0
      evaluation = evaluation + 1
      observation, _ = rec_env.reset()
rec_env.close()
score

Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4
Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4
Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4


[500.0, 500.0]

In [26]:
show_video(num=0)